In [2]:
# !uv pip install kagglehub more_itertools torch transformers logits-processor-zoo
import time

start = time.time()

In [3]:
import pandas as pd
import os

def is_submission():
    return os.getenv("KAGGLE_IS_COMPETITION_RERUN")

path = f"/kaggle/input/jigsaw-agile-community-rules/{'test.csv' if is_submission() else 'test.csv'}"

In [4]:
data = pd.read_csv(path)
data.head(1)

,row_id,body,rule,subreddit,positive_example_1,positive_example_2,negative_example_1,negative_example_2
0,2029,NEW RAP GROUP 17. CHECK US OUT https://soundcl...,"No Advertising: Spam, referral links, unsolici...",hiphopheads,"Hey, guys, just wanted to drop in and invite y...",Cum Swallowing Hottie Katrina Kaif Cartoon Xvi...,SD Stream Eng - [Chelsea TV USA](http://soccer...,HD Streams: |[ENG HD Stoke vs Manchester Unite...


### Steps for initial solution

- Step - 1 Add a column for expanding the rule by describing the given rule and make it understandable along with the prompt.
- Step - 2 Each row updated with prompt with positive and negative examples. 
- Step - 3 Then finally setup system prompt by describing the nature of the tool and get the otuput as voilation or nonvoilation along with other information.
            

In [5]:
import kagglehub
import more_itertools
import torch
from transformers import AutoTokenizer
from transformers import AutoModelForCausalLM

In [6]:
hugging_face_model_name="google/gemma-3/transformers/gemma-3-1b-it"
GEMMA_PATH = kagglehub.model_download(hugging_face_model_name)
processor = AutoTokenizer.from_pretrained(GEMMA_PATH)
device = "cuda" if torch.cuda.is_available() else "cpu"
model = AutoModelForCausalLM.from_pretrained(GEMMA_PATH).to(device)

/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:2225: UnsupportedFieldAttributeWarning: The 'repr' attribute with value False was provided to the `Field()` function, which has no effect in the context it was used. 'repr' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` statement was used, or if the `Field()` function was attached to a single member of a union type.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:2225: UnsupportedFieldAttributeWarning: The 'frozen' attribute with value True was provided to the `Field()` function, which has no effect in the context it was used. 'frozen' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` 

In [7]:
def expand_rules(rule, model, processor, device):

    SYS_PROMPT_RULE = """
    You are an expert with understanding of community guidlines and related keywords.
    For a given community rule provided, you go through its keywords and exapand each keyword in details. 
    
    Output should only contain the expanded rules no explanatory text.
    """
    
    USER_PROMPT_RULE = f"""
    Expand below rule
    
    {rule}
    
    """
    
    prompt_rule_text = (
        f"<start_of_turn>system\n{SYS_PROMPT_RULE}<end_of_turn>\n"
        f"<start_of_turn>user\n{USER_PROMPT_RULE}<end_of_turn>\n"
        f"<start_of_turn>model\n"
    )

    rule_inputs = processor(prompt_rule_text, return_tensors="pt").to(device)
    
    output_rules = model.generate(
        **rule_inputs,
        max_new_tokens=300,
        repetition_penalty=1,
        eos_token_id=processor.eos_token_id,
        pad_token_id=processor.eos_token_id,
        return_dict_in_generate=True
    )
    
    generated_tokens = output_rules.sequences[:, rule_inputs["input_ids"].shape[-1]:]
    decoded_text = processor.batch_decode(generated_tokens, skip_special_tokens=True)

    return decoded_text[0].split("--")[0]
        

In [8]:
import re
rule_df = data[['rule']].drop_duplicates().reset_index(drop=True)
rule_df['expanded_rule'] = [
    re.sub(r'\n{3,}', "", expand_rules(rule, model, processor, device)) 
    for rule in rule_df.rule
]

W1009 12:48:11.947000 37 torch/_inductor/utils.py:1137] [0/0] Not enough SMs to use max_autotune_gemm mode


In [9]:
data = data.merge(rule_df, on="rule", how="left")

In [11]:
data.head(2)

,row_id,body,rule,subreddit,positive_example_1,positive_example_2,negative_example_1,negative_example_2,expanded_rule
0,2029,NEW RAP GROUP 17. CHECK US OUT https://soundcl...,"No Advertising: Spam, referral links, unsolici...",hiphopheads,"Hey, guys, just wanted to drop in and invite y...",Cum Swallowing Hottie Katrina Kaif Cartoon Xvi...,SD Stream Eng - [Chelsea TV USA](http://soccer...,HD Streams: |[ENG HD Stoke vs Manchester Unite...,"No Advertising: Spam, referral links, unsolici..."
1,2030,Make your life comfortable. Get up to 15% Disc...,No legal advice: Do not offer or request legal...,AskReddit,Get a lawyer and get the security camera foota...,That isn't drastic. You tried reaching out to ...,So what are you going to do with the insurance...,It's just for Austria & Germany. If you still ...,No legal advice: Do not offer or request legal...


In [18]:
import torch
import math

def classify_comments_in_batch(df, model, processor, device, batch_size=4):


    token_ids = [processor.get_vocab()[word] for word in ['True', 'False']]
    if any(token_id == processor.get_vocab()['<unk>'] for token_id in token_ids):
          raise ValueError('One of the target classes is not in the vocabulary.')

    responses = []
    num_batches = math.ceil(len(data) / batch_size)

    for i in range(num_batches):
        
        batch = df.iloc[i*batch_size:(i+1)*batch_size]
        prompts = []

        for _, data_row in batch.iterrows():
    
            SYSTEM_PROMPT_CLASSIFICATION = """
            
            you are a community rule voilation inspector. 
            if you are given the rule with context and target comment along with negative and positive comments you can decide if the target comment is in voilation of the specific rule.
            you will use positive comment to understand how those comments voilates the rule and negative comments to understand why negative comments are not breaking the rules,
            you are required to answer only one of the 2 values, 0 or 1. 0 denotes "not voilation" and 1 denotes "voilation".
            
            output should be given between 0 or 1. and strictly float.
            
            """
            
            USER_PROMPT_CLASSIFICATION = f"""
            Classify the below comment based on the rule given, 
            Rule : {data_row.expanded_rule}
            comment = {data_row.body}
            
            below are the positive comment (voilation) and negative comment (non voilation) related to the comment, 
            use these commnet to better understand the above comment: 
            
            positive comment : {data_row.positive_example_1, data_row.positive_example_2} rule violation = 1
            negative comment : {data_row.negative_example_1, data_row.negative_example_2} rule violation = 0
            
            """
        
            prompt_classify_text = (
                f"<start_of_turn>system\n{SYSTEM_PROMPT_CLASSIFICATION}<end_of_turn>\n"
                f"<start_of_turn>user\n{USER_PROMPT_CLASSIFICATION}<end_of_turn>\n"
                f"<start_of_turn>model\n"
            )
            
            prompts.append(prompt_classify_text)

        classifier_inputs = processor(prompts, return_tensors="pt", padding=True, truncation=True).to(device)

        with torch.no_grad():

            outputs = model(**classifier_inputs)
            logits = outputs.logits[:, -1, token_ids]
            probabilities = torch.softmax(logits, dim=-1)
            responses.extend(probabilities[:, 0].tolist())  
            # output_classify = model.generate(
            #     **classifier_inputs,
            #     max_new_tokens=1,
            #     repetition_penalty=1.3,
            #     eos_token_id=processor.eos_token_id,
            #     pad_token_id=processor.eos_token_id,
            #     return_dict_in_generate=True
            # )
    
        # generated_tokens = output_classify.sequences[:, classifier_inputs["input_ids"].shape[-1]:]
        # decoded_text = processor.batch_decode(generated_tokens, skip_special_tokens=True)

        # results.extend([d.split("--")[0] for d in decoded_text])
        # del classifier_inputs, output_classify, generated_tokens
        # torch.cuda.empty_cache()
        
    return responses

In [19]:
pred_proba = classify_comments_in_batch(data, model, processor, device, batch_size=4)

In [20]:
len(pred_proba)

10

In [21]:
df =  data[['row_id']].copy()
df['rule_violation'] = pred_proba
df.to_csv("submission.csv", index=False)

In [22]:
print(time.time() - start)

290.412171125412
